# Real Estate Data Cleaning

In this project, I used messy data to practice data cleaning skills.

The data is about real-estate in Surat, India. 

The data contains nulls, typos and useless infos, for which I used Pandas to get rid of.


## Import and reading input

In [2]:
import pandas as pd

df = pd.read_csv("surat_uncleaned.csv")
df.head(10)

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644 sqft,New Property,Poss. by Oct '24,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...","₹2,891 per sqft",₹33.8 Lac
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278 sqft,New Property,Poss. by Jan '26,6 out of 14,Unfurnished,South -West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,"₹3,551 per sqft",₹45.4 Lac
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173 sqft,Resale,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,"₹3,800 per sqft",₹44.6 Lac
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700 sqft,New Property,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,"₹3,966 per sqft",₹47 Lac
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250 sqft,Orchid Fantasia,New Property,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...","₹3,600 per sqft",₹45 Lac
5,"2 BHK Apartment for Sale in Anand Aspire, Jaha...",Super Area,1265 sqft,Unfurnished,Poss. by Dec '25,New Property,Anand Aspire,2,Multistorey Apartment for Sale in Jahangirabad...,"₹3,411 per sqft",₹43.2 Lac
6,3 BHK Villa for Sale in Dindoli Surat,Carpet Area,1404 sqft,Unfurnished,Poss. by Jun '24,New Property,East,Main Road,contraction,NaN,₹42.1 Lac
7,Office Space for Sale in Vesu Surat,Carpet Area,700 sqft,New Property,Poss. by Sep '25,7 out of 10,1,NaN,"best commercial for business ,investors ,busin...","₹3,392 per sqft",₹44.1 Lac
8,"2 BHK Apartment for Sale in Orchid Gardenia, P...",Super Area,1180 sqft,Orchid Gardenia,New Property,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...","₹3,751 per sqft",₹44.3 Lac
9,2 BHK Apartment for Sale in Palanpur Surat,Carpet Area,720 sqft,New Property,Poss. by Dec '25,3 out of 14,Unfurnished,East,"Irrespective of the business branch, the our g...","₹3,200 per sqft",₹40 Lac


## Null values

A quick check, which columns contain nulls

In [3]:
na_count = df.isna().sum().reset_index()
na_count.columns = ["Column Name", "Na Count"]
na_count

,Column Name,Na Count
0,property_name,0
1,areaWithType,0
2,square_feet,0
3,transaction,104
4,status,1
5,floor,45
6,furnishing,340
7,facing,589
8,description,1371
9,price_per_sqft,368


## Cleaning up 'Transaction' column

- This column contains both nulls and useless information

- If it's not "New Property" or "Resale", then "Other" will suffice

In [6]:
df_trans = df.copy()
df_trans.loc[~df_trans["transaction"].isin(["New Property", "Resale"]), "transaction"] = "Other"
pd.unique(df_trans["transaction"])

array(['New Property', 'Resale', 'Other'], dtype=object)

## Cleaning up 'Status' column

- The original data read the dates for contracted flats (e.g. "Poss. by Dec. 5"), which we do not need. For these, I replaced them with "Contracted"

- The original data contained values like "20 X 50.2" (the area) or "3 out of 10" (the floor). For these, I replaced them with "For Sale"

- There were also values, supposedly indicating the owners. For these, I replaced them with "Sold"

In [7]:
df_stat = df_trans.copy()

df_stat.loc[df_stat["status"].str.contains("Poss", na=False), "status"] = "Contracted"
df_stat.loc[df_stat["status"].str.contains("X", na=False), "status"] = "For Sale"
df_stat.loc[df_stat["status"].str.contains("out of", na=False), "status"] = "For Sale"
df_stat.loc[df_stat["status"].isin(["Co-operative Society", "2", "Const. Age New Construction"]), "status"] = "Sold"
df_stat.loc[df_stat["status"].isna(), "status"] = "For Sale"
pd.unique(df_stat["status"])

array(['Contracted', 'Ready to Move', 'New Property', 'For Sale',
       'Freehold', 'Resale', 'Sold', 'Power Of Attorney'], dtype=object)

## Cleaning up 'Floor' column

- I created two new columns ('Flat floor' and 'All floors')

- There is a different case for ground or basement flats.

- I dropped the original 'Floor' column, as there is no use for it 

In [11]:
df_flo = df_stat.copy()

df_flo.loc[df_flo["floor"].isna(), "floor"] = "Unknown"

df_flo['flat_floor'] = df_flo['floor'].str.extract(r'(\d+)', expand=False)
df_flo.loc[df_flo["floor"].str.contains("Basement"), "flat_floor"] = "Basement"
df_flo.loc[df_flo["floor"].str.contains("Ground"), "flat_floor"] = "Ground"
df_flo.loc[df_flo["flat_floor"].isna(), "flat_floor"] = "Unknown"

df_flo['all_floors'] = df_flo['floor'].str.extract(r'out of\s*(\d+)', expand=False)
df_flo.loc[df_flo["all_floors"].isna(), "all_floors"] = "Unknown"
df_flo.drop(columns="floor", inplace=True)

# df_flo

## Cleaning up 'Furnishing' column

- This column had a lot of useless data. So, if the value isn't "Unfurnished", "Semi-Furnished" or "Furnished", I replaced them with "Unknown"


In [25]:
df_fur = df_flo.copy()

df_fur.loc[~df_fur["furnishing"].isin(["Unfurnished", "Semi-Furnished", "Furnished"]), "furnishing"] = "Unknown" 

## Cleaning up 'Facing' column

- Again, getting rid of useless data

- I used 'str.contains', because there are values for intercardinal directions

- Checking the unique values, there's a small typo for "South - West"

In [26]:
df_fac = df_fur.copy()

df_fac.loc[~df_fac["facing"].str.contains(r'\b(North|South|East|West)\b', na=False), "facing"] = "Unknown"
df_fac.loc[df_fac["facing"] == "South -West", "facing"] = "South - West"

# pd.unique(df_fac["facing"])


C:\Users\antal\AppData\Local\Temp\ipykernel_14348\3871741804.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_fac.loc[~df_fac["facing"].str.contains(r'\b(North|South|East|West)\b', na=False), "facing"] = "Unknown"


## Dropping the 'Description' column

- The 'Luxury' column requires no cleaning

In [27]:
df_lux = df_fac.drop(columns="description")

## Cleaning up 'Price Per Square Feet' column

In [28]:
df_pps = df_lux.copy()

df_pps.loc[df_pps["price_per_sqft"].isna(), "price_per_sqft"] = "Unknown"

## Cleaning up 'Area With Type' column

- I also dropped the 'Property Name' column

In [29]:
df_pri = df_pps.copy()

df_pri.loc[df_pri["areaWithType"].isin(["Transaction", "Status"]), "areaWithType"] = "Unknown"

df_pri.drop(columns="property_name", inplace=True)


## Cleaning up 'Price' and 'Square Feet' column

- I got rid of the currency symbol for indian rubel, using regex

- There were values, which read "Call For Price". For these, I replaced them with "Unknown"

- I used a function to convert indian rubels to USD

- I got rid of the units of measures in the 'Square Feet' column

- I created new columns: "sqft_clean" and "price_sqft_usd"

- Finally, I dropped the original, uncleaned columns


In [34]:
df_usd = df_pri.copy()

df_usd["price_clean"] = df_usd["price"].str.replace("₹", "", regex=False).str.strip()

df_usd.loc[df_usd["price_clean"].str.contains("Call"), "price_clean"] = "Unknown"

def inr_to_usd(val, rate=83):
    if (pd.isna(val) or val=="Unknown"):
        return None
    val = val.replace(",", "")
    if "Lac" in val:
        num = float(val.replace("Lac", "").strip()) * 1e5
    elif "Cr" in val:
        num = float(val.replace("Cr", "").strip()) * 1e7
    else:
        num = float(val)

    return round(num / rate, 2)
        

df_usd["price_usd"] = df_usd["price_clean"].apply(inr_to_usd)

df_usd["square_feet"] = df_usd["square_feet"].str.replace("sqft", "", regex=False).str.strip()
df_usd["square_feet"] = df_usd["square_feet"].str.replace("sqm", "", regex=False).str.strip()
df_usd["square_feet"] = df_usd["square_feet"].str.replace("sqyrd", "", regex=False).str.strip()
df_usd.loc[df_usd["square_feet"].isin(["Resale"]), "square_feet"] = "Unknown"

In [36]:
df_usd["sqft_clean"] = (df_usd["square_feet"]
                       .astype(str)
                       .str.extract(r'(\d+\.?\d*)')[0]
                       .astype(float))

df_usd["price_sqft_usd"] = round(df_usd["price_usd"] / df_usd["sqft_clean"].astype(float), 2)

In [37]:
df_usda = df_usd.drop(columns=["square_feet", "price_per_sqft", "price", "price_clean"], inplace=False)
df_usda

,areaWithType,transaction,status,furnishing,facing,flat_floor,all_floors,price_usd,sqft_clean,price_sqft_usd
0,Carpet Area,New Property,Contracted,Unfurnished,West,5,10,40722.89,644.0,63.23
1,Super Area,New Property,Contracted,Unfurnished,South - West,6,14,54698.80,1278.0,42.80
2,Super Area,Resale,Ready to Move,Semi-Furnished,East,5,13,53734.94,1173.0,45.81
3,Carpet Area,New Property,Ready to Move,Unfurnished,East,6,14,56626.51,700.0,80.90
4,Super Area,Other,New Property,Unknown,Unknown,Unknown,Unknown,54216.87,1250.0,43.37
...,...,...,...,...,...,...,...,...,...,...
4520,Carpet Area,New Property,Contracted,Unfurnished,South - East,5,12,NaN,2000.0,NaN
4521,Super Area,New Property,Contracted,Unfurnished,South - East,5,16,NaN,3600.0,NaN
4522,Carpet Area,New Property,Contracted,Unfurnished,North - East,7,13,NaN,2250.0,NaN
4523,Carpet Area,New Property,Ready to Move,Unfurnished,North - West,7,18,NaN,3450.0,NaN


## Exporting clean data

In [38]:
df_cleaned = df_usda.copy()

df_cleaned.to_csv("cleaned.csv")